In [1]:
import sympy 
import symengine as se
import constitutive_data
import numpy as np

In [2]:
def scalar_derivation_wrt_matrix(sympy_expression, matrix, dim=3):
    P = sympy.zeros(dim)
    for ii in range(dim):
        for jj in range(dim):
            P[ii,jj] = sympy.diff(sympy_expression,matrix[ii,jj])
    P_matrix = sympy.Matrix(P)
    return P_matrix

In [3]:
nodes,adj = constitutive_data.sample_tree()

In [4]:
def to_algebraic_string(nodes, adj, i = 0):
    if list(',') in list(map(str.split, nodes[i])):
        return list(map(str.split, str(nodes[i])))[0][0] + '.'+ list(map(str.split, str(nodes[i])))[2][0]
    if nodes[i] == '+' or nodes[i] == '*' or nodes[i] == '-'  or nodes[i] == '/':
        return to_algebraic_string(nodes, adj, adj[i][0]) + ' ' + nodes[i] + ' ' + to_algebraic_string(nodes, adj, adj[i][1])
    if nodes[i] == 'log' or nodes[i] == 'exp':
        return nodes[i] + '(' + to_algebraic_string(nodes, adj, adj[i][0]) + ')'
    if nodes[i] == 'pow':
        return '(' + to_algebraic_string(nodes, adj, adj[i][0]) + ')' + '**' + to_algebraic_string(nodes, adj, adj[i][1])
    if nodes[i] == '(J-1)':
        return '(' + nodes[i] + ')' 
    if nodes[i] == '(I1-3)':
        return '(' + 'I1-3' + ')' 
    if nodes[i] == '(I2-3)':
        return '(' + 'I2-3' + ')'
    else:
        return nodes[i]

In [36]:
def stress_correction(sympy_expression, F, J, dim=3):
    dWdF = sympy.diff(sympy_expression, J)
    return dWdF

def get_J(dim=3):
    return sympy.Symbol("J")
    
def get_J_wrt_F(dim=3):
    return sympy.Determinant(sympy.MatrixSymbol("F",dim,dim).as_explicit())

def get_J_wrt_C(F):
    return sympy.Pow(sympy.Determinant(F.T*F),0.5)

def get_I1(dim=3):
    return sympy.Symbol("I1")

def get_I1_wrt_C(F):
    return sympy.Pow(sympy.Determinant(F),-2/3)*sympy.Trace(F.T*F)

def get_I2(dim=3):
    return sympy.Symbol("I2")

def get_I2_wrt_C(F):
    return sympy.Pow(sympy.Determinant(F),-4/3)*(0.5*(sympy.Pow(sympy.Trace(sympy.MatMul(F.T,F)),2) - sympy.Trace(sympy.MatPow(sympy.MatMul(F.T,F),2))))

In [37]:
J, I1, I2 = se.symbols("J I1 I2")
F11, F12, F21, F22 = se.symbols("F11 F12 F21 F22")
F = sympy.MutableDenseMatrix([[F11, F12, 0],[F21, F22, 0],[0,0,1]])
sympy_expression = constitutive_data.to_sympy_expression(to_algebraic_string(nodes,adj))

In [38]:
sympy_expression

2*I1 + J + 1.16

In [50]:
J_simp = sympy.simplify(get_J_wrt_C(F))
I1_simp = sympy.simplify(get_I1_wrt_C(F))
I2_simp = sympy.simplify(get_I2_wrt_C(F))

In [51]:
expression_wrtF = sympy_expression.subs({J:J_simp, I1:I1_simp})
expression_wrtF

2*(F11**2 + F12**2 + F21**2 + F22**2 + 1)/(F11*F22 - F12*F21)**0.666666666666667 + (F11**2*F22**2 - 2*F11*F12*F21*F22 + F12**2*F21**2)**0.5 + 1.16

In [47]:
simplified = se.sympify(expression_wrtF)

In [57]:
se.diff(expression_wrtF,F22)

4*(F11*F22 - F12*F21)**(-0.666666666666667)*F22 + 0.5*(2*F11**2*F22 - 2*F11*F12*F21)*(F11**2*F22**2 + F12**2*F21**2 - 2*F11*F12*F21*F22)**(-0.5) - 1.33333333333333*(1 + F11**2 + F12**2 + F21**2 + F22**2)*(F11*F22 - F12*F21)**(-1.66666666666667)*F11

In [ ]:
def compute_invariants():
    F11, F12, F21, F22 = se.symbols("F11 F12 F21 F22")
    F = sympy.MutableDenseMatrix([[F11, F12, 0],[F21, F22, 0],[0,0,1]])
    J  = sympy.Pow(sympy.Determinant(F.T*F),0.5)
    I1 = sympy.Pow(sympy.Determinant(F),-2/3)*sympy.Trace(F.T*F)
    I2 = sympy.Pow(sympy.Determinant(F),-4/3)*(0.5*(sympy.Pow(sympy.Trace(sympy.MatMul(F.T,F)),2) - sympy.Trace(sympy.MatPow(sympy.MatMul(F.T,F),2))))
    return sympy.simplify(J), sympy.simplify(I1), sympy.simplify(I2), F11, F12, F21, F22


Ji, I1i, I2i, F11i, F12i, F21i, F22i = compute_invariants()

def compute_loss(F11, F12, F21, F22, W, num_nodes_per_element, numNodes,\
                 voigt_map, gradNa, qpWeights, connectivity, dirichlet_nodes, \
                 reactions):

    # Get gradients of W w.r.t F (compute)
    dW_NN_dF11 = se.diff(W,F11)
    dW_NN_dF12 = se.diff(W,F12)
    dW_NN_dF21 = se.diff(W,F21)
    dW_NN_dF22 = se.diff(W,F22)

    P11 = np.zeros((numNodes,))
    P21 = np.zeros((numNodes,))
    P12 = np.zeros((numNodes,))
    P22 = np.zeros((numNodes,))
    for i in range(0, numNodes):
        P11[i] = np.array(dW_NN_dF11.subs({F11i:F11[i], F12i:F12[i], F21i:F21[i], F22i:F22[i]}).evalf(16)).astype(np.float128)
        P12[i] = np.array(dW_NN_dF12.subs({F11i:F11[i], F12i:F12[i], F21i:F21[i], F22i:F22[i]}).evalf(16)).astype(np.float128)
        P21[i] = np.array(dW_NN_dF21.subs({F11i:F11[i], F12i:F12[i], F21i:F21[i], F22i:F22[i]}).evalf(16)).astype(np.float128)
        P22[i] = np.array(dW_NN_dF22.subs({F11i:F11[i], F12i:F12[i], F21i:F21[i], F22i:F22[i]}).evalf(16)).astype(np.float128)

    # Assemble First Piola-Kirchhoff stress components
    P = np.concatenate((P11[:,None],P12[:,None],P21[:,None], P22[:,None]),dim=1)

    # compute internal forces on nodes
    f_int_nodes = np.zeros(numNodes,2)
    for a in range(num_nodes_per_element):
        for i in range(2):
            for j in range(2):
                force = P[:,voigt_map[i][j]] * gradNa[a][:,j] * qpWeights
                f_int_nodes[:,i].add.at(connectivity[a],0,force)

    f_int_nodes_clone = f_int_nodes.clone()
    f_int_nodes_clone[dirichlet_nodes] = 0.
    eqb_loss = np.sum(f_int_nodes_clone**2)

    reaction_loss = np.tensor([0.])
    for reaction in reactions:
        reaction_loss += (np.sum(f_int_nodes[reaction.dofs]) - reaction.force)**2
    loss += eqb_loss +  reaction_loss
    return loss